In [3]:
from src import util
import tensorflow as tf
import numpy as np

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
sensor_data_trainval, sequence_ids_trainval, labels_trainval = util.ingest_sensor_data('train.csv', 'train_labels.csv')
sensor_data_test, sequence_ids_test, _ = util.ingest_sensor_data('test.csv')

sensor_data_train, sequence_ids_train, labels_train, sensor_data_val, sequence_ids_val, labels_val = util.validation_split(training_fraction=0.7, sensor_data=sensor_data_trainval, sequence_ids=sequence_ids_trainval, labels=labels_trainval)

(25968, 60, 13) 25968
(12218, 60, 13) 12218


In [19]:
kernel_size=5
filters=100
dense_dim=32

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.3, epochs=50)

Epoch 1/50
569/569 [==============================] - 2s 3ms/step - loss: 0.6669 - accuracy: 0.5950 - auc: 0.6300 - val_loss: 0.6395 - val_accuracy: 0.6524 - val_auc: 0.6908
Epoch 2/50
569/569 [==============================] - 2s 3ms/step - loss: 0.6295 - accuracy: 0.6523 - auc: 0.6992 - val_loss: 0.6103 - val_accuracy: 0.6681 - val_auc: 0.7237
Epoch 3/50
569/569 [==============================] - 2s 3ms/step - loss: 0.5987 - accuracy: 0.6809 - auc: 0.7372 - val_loss: 0.5967 - val_accuracy: 0.6936 - val_auc: 0.7610
Epoch 4/50
569/569 [==============================] - 2s 3ms/step - loss: 0.5722 - accuracy: 0.7015 - auc: 0.7660 - val_loss: 0.5630 - val_accuracy: 0.7219 - val_auc: 0.7886
Epoch 5/50
569/569 [==============================] - 2s 3ms/step - loss: 0.5414 - accuracy: 0.7274 - auc: 0.7975 - val_loss: 0.5353 - val_accuracy: 0.7326 - val_auc: 0.8036
Epoch 6/50
569/569 [==============================] - 2s 3ms/step - loss: 0.5174 - accuracy: 0.7468 - auc: 0.8182 - val_loss: 0.52

In [20]:
pred = model.predict(util.preprocess_sensor_data(sensor_data_test))

In [21]:
pred.shape

(12218, 1)

In [22]:
np.unique(pred)

array([1.7913841e-08, 4.8177881e-08, 6.1597305e-08, ..., 9.9988347e-01,
       9.9990582e-01, 9.9994326e-01], dtype=float32)

In [23]:
util.generate_submission(pred, sequence_ids_test, 'conv1d-reg')

In [24]:
# two convolution layers

kernel_size=5
filters=43
dense_dim=32

model2 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model2.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.3, epochs=50)

pred2 = model2.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred2, sequence_ids_test, 'conv1d-2layer')

Epoch 1/50
569/569 [==============================] - 3s 4ms/step - loss: 0.6690 - accuracy: 0.5846 - auc: 0.6213 - val_loss: 0.6300 - val_accuracy: 0.6617 - val_auc: 0.7192
Epoch 2/50
569/569 [==============================] - 2s 4ms/step - loss: 0.5942 - accuracy: 0.6877 - auc: 0.7487 - val_loss: 0.5354 - val_accuracy: 0.7344 - val_auc: 0.8154
Epoch 3/50
569/569 [==============================] - 2s 4ms/step - loss: 0.5216 - accuracy: 0.7432 - auc: 0.8201 - val_loss: 0.5158 - val_accuracy: 0.7423 - val_auc: 0.8422
Epoch 4/50
569/569 [==============================] - 2s 4ms/step - loss: 0.4815 - accuracy: 0.7738 - auc: 0.8508 - val_loss: 0.4886 - val_accuracy: 0.7520 - val_auc: 0.8619
Epoch 5/50
569/569 [==============================] - 2s 4ms/step - loss: 0.4651 - accuracy: 0.7808 - auc: 0.8620 - val_loss: 0.4521 - val_accuracy: 0.7830 - val_auc: 0.8755
Epoch 6/50
569/569 [==============================] - 2s 4ms/step - loss: 0.4460 - accuracy: 0.7927 - auc: 0.8744 - val_loss: 0.43

In [25]:
pred2 = model2.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred2, sequence_ids_test, 'conv1d-2layer')

In [30]:
# three convolution layers

kernel_size=5
filters=64
dense_dim=64

dropout_conv=0.4
dropout_dense=0.5

model3 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(dropout_dense),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model3.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.3, epochs=50)

pred3 = model3.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred3, sequence_ids_test, 'conv1d-3layer')

Epoch 1/50
569/569 [==============================] - 4s 6ms/step - loss: 0.6831 - accuracy: 0.5626 - auc: 0.5835 - val_loss: 0.6762 - val_accuracy: 0.5859 - val_auc: 0.6759
Epoch 2/50
569/569 [==============================] - 3s 5ms/step - loss: 0.6234 - accuracy: 0.6584 - auc: 0.7113 - val_loss: 0.5899 - val_accuracy: 0.7312 - val_auc: 0.8079
Epoch 3/50
569/569 [==============================] - 3s 5ms/step - loss: 0.5435 - accuracy: 0.7293 - auc: 0.8041 - val_loss: 0.5214 - val_accuracy: 0.7679 - val_auc: 0.8524
Epoch 4/50
569/569 [==============================] - 3s 6ms/step - loss: 0.5012 - accuracy: 0.7613 - auc: 0.8376 - val_loss: 0.4816 - val_accuracy: 0.7919 - val_auc: 0.8736
Epoch 5/50
569/569 [==============================] - 3s 6ms/step - loss: 0.4788 - accuracy: 0.7757 - auc: 0.8544 - val_loss: 0.4604 - val_accuracy: 0.8007 - val_auc: 0.8837
Epoch 6/50
569/569 [==============================] - 3s 6ms/step - loss: 0.4587 - accuracy: 0.7879 - auc: 0.8674 - val_loss: 0.44